In [ ]:
from IMPORTS import *
import pandas as pd
seed_value = 18
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

c:\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and 

In [ ]:
ft = fasttext.load_model('WordEmbeddings/cc.en.300.bin')

# ATE

Reading ATE json files

In [ ]:
ATE_train = None
ATE_test = None
ATE_val = None
with open("Task1\processed\ATE_train.json") as f1:
    ATE_train  = json.load(f1)

with open("Task1\processed\ATE_test.json") as f2:
    ATE_test = json.load(f2)

with open("Task1\processed\ATE_val.json") as f3:
    ATE_val = json.load(f3)

ATE Dataset

In [ ]:
X_train_ate,Y_train_ate= create_train_test_val(ATE_train)
X_test_ate,Y_test_ate = create_train_test_val(ATE_test)
X_val_ate,Y_val_ate = create_train_test_val(ATE_val)

Tokenizing the ATE Dataset

In [ ]:
X_train_ate_tokenized, toke = TokenCreator(X_train_ate,1)
X_test_ate_tokenized = TokenCreator(X_test_ate,0,tokenizer = toke)
X_val_ate_tokenized = TokenCreator(X_val_ate,0,tokenizer = toke)

Y_train_ate_tokenized,toke2 = TokenCreator(Y_train_ate,0)
Y_test_ate_tokenized = TokenCreator(Y_test_ate,0,tokenizer = toke2)
Y_val_ate_tokenized = TokenCreator(Y_val_ate,0,tokenizer = toke2)

Vocabulary of ATE

In [ ]:
st1 = find_vocab(X_train_ate_tokenized)
vocab_size_ate  = len(st1) + 1 
vocab_size_ate

2406

Padding ATE

In [ ]:
X_padded_train = pad_sequences(X_train_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_train = pad_sequences(Y_train_ate_tokenized, maxlen=100, padding="pre", truncating="post")

X_padded_test = pad_sequences(X_test_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_test = pad_sequences(Y_test_ate_tokenized, maxlen=100, padding="pre", truncating="post")

X_padded_val = pad_sequences(X_val_ate_tokenized, maxlen=100, padding="pre", truncating="post")
Y_padded_val = pad_sequences(Y_val_ate_tokenized, maxlen=100, padding="pre", truncating="post")

One hot encoding the ATE labels

In [ ]:
Y_padded_train = to_categorical(Y_padded_train)
Y_padded_test = to_categorical(Y_padded_test)
Y_padded_val = to_categorical(Y_padded_val)

Assigning **ATE Fasttext** mapping

In [ ]:
weights_ate_ft = np.zeros((vocab_size_ate, 300))

mapping_ate_ft = toke.word_index 
mapping_ate_ft

Fetching weights from pre trained Fasttext model (ATE)

In [ ]:
for word, index in mapping_ate_ft.items():
    if word in ft:
        weights_ate_ft[index:] = ft[word]

Initializing the rnn Model using Fasttext embeddings

In [ ]:
rnn_model_ft = Sequential()
rnn_model_ft.add(Embedding(input_dim = vocab_size_ate,output_dim = 300,input_length  = 100,weights = [weights_ate_ft],trainable = False))
rnn_model_ft.add(SimpleRNN(64, return_sequences=True))
rnn_model_ft.add(TimeDistributed(Dense(Y_padded_train.shape[2], activation='softmax')))

Compiling the rnn Fasttext Model

In [ ]:
rnn_model_ft.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc',Precision(),Recall()])

In [ ]:
rnn_f1_callback_val = F1ScoreCallback(data=(X_padded_val, Y_padded_val),name='validation')
rnn_f1_callback_train=F1ScoreCallback(data=(X_padded_train, Y_padded_train),name='training')

In [ ]:
rnn_model_ft.summary()

Fitting the rnn model

In [ ]:
rnn_training_ft = rnn_model_ft.fit(X_padded_train, Y_padded_train, batch_size=128, epochs=15, validation_data=(X_padded_val, Y_padded_val), callbacks=[rnn_f1_callback_train,rnn_f1_callback_val])

In [ ]:
accuracy = rnn_model_ft.evaluate(X_padded_test, Y_padded_test, verbose = 1)
print(accuracy[1])

In [ ]:
Y_pred_ate = rnn_model_ft.predict(X_padded_test)
real = get_real(Y_padded_test)
pred = get_pred(Y_pred_ate)
f1 = F1Score(average='macro')
f1.update_state(real,pred)
f1_score = f1.result().numpy()
print(f"F1 Score ATE : {f1_score}")

Plotting the graph

In [ ]:
draw_loss_graph(rnn_training_ft, "rnn Model", rnn_f1_callback_train.f1_scores, rnn_f1_callback_val.f1_scores)
save_model(rnn_model_ft, "rnn","FastText","t2")
loaded_model = load_model("Saved_Models/ATE/t2_rnn_FastText.pkl")